In [ ]:
import os
import torch

# Configuration
IMAGE_SIZE = 224
BATCH_SIZE = 32
EPOCHS = 20
DEVICE = torch.device("cuda" if torch.cuda.is_available() else "cpu")
# DATA_DIR = '/kaggle/input/utkface-new/crop_part1' # Update this path!
DATA_DIR = '/kaggle/input/utkface-uncropped-dataset/Images'

print(DEVICE)


In [ ]:
!pip install retina-face

In [ ]:
import matplotlib.pyplot as plt
import cv2
from retinaface import RetinaFace
faces = RetinaFace.extract_faces(img_path = "/kaggle/input/utkface-uncropped-dataset/Images/29_0_1_20170113154817089.jpg", align=True)

for face in faces:
  plt.imshow(face)
  plt.show()

faces_2 = RetinaFace.detect_faces("/kaggle/input/utkface-uncropped-dataset/Images/29_0_1_20170113154817089.jpg")
# print(faces_2['face_1']['facial_area'])
print(type(faces_2))

img = cv2.imread("/kaggle/input/utkface-uncropped-dataset/Images/29_0_1_20170113154817089.jpg")
plt.imshow(img)

In [ ]:
import os
import cv2
import numpy as np
import pandas as pd
from tqdm import tqdm
from retinaface import RetinaFace
from PIL import Image
import math

def align_face(img, left_eye, right_eye):
    """
    Rotates the image to make the line between eyes horizontal.
    Returns: rotated_img, rotation_matrix, new_center
    """
    # 1. Calculate angle
    dY = right_eye[1] - left_eye[1]
    dX = right_eye[0] - left_eye[0]
    angle = np.degrees(np.arctan2(dY, dX)) 
    
    # 2. Get rotation matrix
    h, w = img.shape[:2]
    center = (w // 2, h // 2)
    M = cv2.getRotationMatrix2D(center, angle, 1.0)
    
    # 3. Rotate Image
    # Calculate new bounding dimensions to avoid cropping after rotation
    cos = np.abs(M[0, 0])
    sin = np.abs(M[0, 1])
    new_w = int((h * sin) + (w * cos))
    new_h = int((h * cos) + (w * sin))
    
    # Adjust matrix translation
    M[0, 2] += (new_w / 2) - center[0]
    M[1, 2] += (new_h / 2) - center[1]
    
    rotated_img = cv2.warpAffine(img, M, (new_w, new_h))
    return rotated_img, M

def transform_point(point, M):
    """Applies affine transform to a single (x, y) point."""
    v = [point[0], point[1], 1]
    transformed = np.dot(M, v)
    return (int(transformed[0]), int(transformed[1]))

In [ ]:
def align_face(img, left_eye, right_eye):
    # left_eye = Person's Left Eye (Image Right)
    # right_eye = Person's Right Eye (Image Left)
    
    # We want vector from Image Left -> Image Right
    # So: Person's Left Eye - Person's Right Eye
    dY = left_eye[1] - right_eye[1]
    dX = left_eye[0] - right_eye[0]
    
    angle = np.degrees(np.arctan2(dY, dX))
    
    # 2. Get rotation matrix
    h, w = img.shape[:2]
    center = (w // 2, h // 2)
    M = cv2.getRotationMatrix2D(center, angle, 1.0)
    
    # 3. Rotate Image
    cos = np.abs(M[0, 0])
    sin = np.abs(M[0, 1])
    new_w = int((h * sin) + (w * cos))
    new_h = int((h * cos) + (w * sin))
    
    M[0, 2] += (new_w / 2) - center[0]
    M[1, 2] += (new_h / 2) - center[1]
    
    rotated_img = cv2.warpAffine(img, M, (new_w, new_h))
    return rotated_img, M

# CROP 3 SIDE -> ROTATE (normal)

In [ ]:
import cv2
import os
import numpy as np
import math
from retinaface import RetinaFace
from tqdm import tqdm
import matplotlib.pyplot as plt

# Configuration
INPUT_DIR = "/kaggle/input/utkface-uncropped-dataset/Images"
OUTPUT_DIR = "/kaggle/working/cropped-utkface"
CSV_OUTPUT_PATH = "/kaggle/working/skipped_images.csv" # <--- CHANGE 2: Define CSV path
MARGIN = 0.2
TARGET_SIZE = (224, 224) 


if not os.path.exists(OUTPUT_DIR):
    os.makedirs(OUTPUT_DIR)
    
# images_all = os.listdir(INPUT_DIR)
# print(len(images_all))

images = [f for f in os.listdir(INPUT_DIR) if f.endswith(('.jpg', '.png', '.jpeg'))]
print(type(images[0]))

bad_images = [
    '1_1_0_20170109190459034.jpg',
    '35_0_1_20170113155220675.jpg',
    '29_0_1_20170113154817089.jpg',
    '27_0_0_20170117175751139.jpg',
    '90_0_2_20170111211411216.jpg',
    '20_0_1_20170117012858687.jpg',
    '28_0_1_20170113180639429.jpg',
    '90_0_2_20170111210008312.jpg',
    '2_0_3_20161220144926901.jpg',
    '12_1_2_20170103201240488.jpg',
]

LIMIT = 0
skipped_files = []

for img_name in tqdm(images):
    # if LIMIT > 10:
    #     break

    img_name = img_name.strip()
    img_path = os.path.join(INPUT_DIR, img_name)
    
    faces = RetinaFace.detect_faces(img_path)
    
    if not faces:
        img_no_faces = cv2.imread(img_path)
        cv2.imwrite(os.path.join(OUTPUT_DIR, img_name), img_no_faces)
        
        skipped_files.append({'filename':img_name})
        continue
        
    face_data = faces['face_1']
    landmarks = face_data['landmarks']
    bbox = face_data['facial_area'] 
    
    # 2. Get Eye Coordinates & Face Center
    left_eye_img = landmarks["left_eye"]   # Person's left eye (on image right)
    right_eye_img = landmarks["right_eye"] # Person's right eye (on image left)
    
    x1, y1, x2, y2 = bbox
    face_w = x2 - x1
    face_h = y2 - y1
    face_cx = x1 + (face_w // 2)
    face_cy = y1 + (face_h // 2)
    
    # 3. Calculate Rotation Angle
    dy = left_eye_img[1] - right_eye_img[1]
    dx = left_eye_img[0] - right_eye_img[0]
    angle = np.degrees(np.arctan2(dy, dx)) 
    
    # ---------------------------------------------------------
    # 4. Calculate Scale (Modified for Top/Side Margins only)
    # ---------------------------------------------------------
    # Width needs margin on Left + Right (2 * MARGIN)
    target_w_box = face_w * (1 + MARGIN * 2)
    
    # Height needs margin on Top ONLY (1 * MARGIN)
    target_h_box = face_h * (1 + MARGIN)
    
    # We choose the max dimension to ensure the face fits inside
    desired_size = max(target_w_box, target_h_box)
    scale = TARGET_SIZE[0] / desired_size
    
    # ---------------------------------------------------------
    # 5. Build the Affine Matrix (Modified to shift face down)
    # ---------------------------------------------------------
    center = (float(face_cx), float(face_cy))
    M = cv2.getRotationMatrix2D(center, angle, scale)
    
    # Standard centering (moves face center to image center)
    t_x = (TARGET_SIZE[0] / 2) - face_cx
    t_y = (TARGET_SIZE[1] / 2) - face_cy
    
    # SHIFT CORRECTION:
    # Because we added margin to Top but NOT Bottom, the logical center 
    # of our crop is higher up on the face. To center that "high point",
    # we must push the face DOWN in the final image.
    # The shift amount is exactly half the margin height in pixels.
    shift_y = (face_h * MARGIN * 0.5) * scale
    
    M[0, 2] += t_x
    M[1, 2] += t_y + shift_y  # Add positive Y to move image down
    
    # 6. Apply Warp
    img = cv2.imread(img_path)
    aligned_face = cv2.warpAffine(img, M, TARGET_SIZE, flags=cv2.INTER_CUBIC, borderMode=cv2.BORDER_CONSTANT, borderValue=(0,0,0))
    
    # Visualization
    # plt.figure(figsize=(10, 5))
    # plt.subplot(1, 2, 1)
    # plt.imshow(cv2.cvtColor(img, cv2.COLOR_BGR2RGB))
    # plt.title("Original")
    
    # plt.subplot(1, 2, 2)
    # plt.imshow(cv2.cvtColor(aligned_face, cv2.COLOR_BGR2RGB))
    # plt.title("Aligned (Top Margin Only)")
    # plt.show()

    cv2.imwrite(os.path.join(OUTPUT_DIR, img_name), aligned_face)
    
    LIMIT += 1

print("IMAGE COUNT : " + str(LIMIT))

df_skipped = pd.DataFrame(skipped_files)
df_skipped.to_csv(CSV_OUTPUT_PATH, index=False)
print(f"Skipped files saved to: {CSV_OUTPUT_PATH}")

print("Preprocessing Complete.")

In [ ]:
import shutil

# Define the source folder path and the desired output zip file name (without the extension)
source_dir = OUTPUT_DIR
output_zip_name = 'utkface-croped-margin20' # This will create 'my_project_archive.zip'

if not os.path.exists(output_zip_name + '.zip'):
    shutil.make_archive(output_zip_name, 'zip', source_dir)    
    print(f"Folder '{source_dir}' successfully zipped to '{output_zip_name}.zip'")
else:
    print(f"file already exist")
